In [1]:
%pylab inline

import numpy as np
import os
import pysumma as ps
import itertools
import xarray as xr
import cycler
from pprint import pprint as pp

user = os.environ['USER']
file_manager = ('/pool0/home/andrbenn/data/workspace/'
                'whw2019_snow_modeling/summa_setup/file_manager.txt')
summa_exe = '/opt/local/bin/summa.exe'
        
decisions_options = {
    'thCondSnow': ['tyen1965', 'melr1977', 'jrdn1991', 'smnv2000'],
    #'compaction': ['consettl', 'anderson'],
    #'snowDenNew': ['hedAndPom', 'anderson', 'pahaut_76'],
    #'snowLayers': ['jrdn1991', 'CLM_2010']
}

param_options = {
}

# Link to where you can find output variables:
#  https://github.com/NCAR/summa/blob/master/build/source/dshare/var_lookup.f90
output_vars_to_add = ['mLayerTemp']
    
#config = ps.ensemble.total_product(decisions_options, param_options)
config = ps.ensemble.decision_product(decisions_options)
stack_dims = list(decisions_options.keys()) + list(param_options.keys())
len(config)

Populating the interactive namespace from numpy and matplotlib


4

In [2]:
ens = ps.Ensemble(summa_exe, file_manager, config, num_workers=24)
#for name, sim in ens.simulations.items():
#    for out_var in output_vars_to_add:
#        sim.output_control.set_option(name=var, period=1)

In [ ]:
ens.start('local', arg_list=['export OMP_NUM_THREADS=1', 
                             'export LD_LIBRARY_PATH=/opt/local/lib'])
ens.monitor()

In [ ]:
for i, (n, s) in enumerate(ens.simulations.items()):
    if s._status != 'Success':
        print(i, f'{n:100}', s._status)

In [10]:
print(ens.simulations['++smnv2000++anderson++hedAndPom++jrdn1991++'].stderr)

*** Error in `/opt/local/bin/summa.exe': free(): invalid next size (normal): 0x000000000243fee0 ***
======= Backtrace: =========
/lib64/libc.so.6(+0x81489)[0x7f29e101c489]
/opt/local/bin/summa.exe[0x462590]
/opt/local/bin/summa.exe[0x4a6627]
/opt/local/bin/summa.exe[0x402b6d]
/lib64/libc.so.6(__libc_start_main+0xf5)[0x7f29e0fbd3d5]
/opt/local/bin/summa.exe[0x402b96]
======= Memory map: ========
00400000-0053f000 r-xp 00000000 09:7d 268837501                          /opt/local/bin/summa.exe
0073e000-0073f000 r--p 0013e000 09:7d 268837501                          /opt/local/bin/summa.exe
0073f000-00773000 rw-p 0013f000 09:7d 268837501                          /opt/local/bin/summa.exe
00773000-007f6000 rw-p 00000000 00:00 0 
01fa6000-02478000 rw-p 00000000 00:00 0                                  [heap]
7f29d4000000-7f29d4021000 rw-p 00000000 00:00 0 
7f29d4021000-7f29d8000000 ---p 00000000 00:00 0 
7f29dbf81000-7f29dbf83000 r-xp 00000000 09:7d 32051                      /usr/lib64/libfr

In [5]:
ds = ens.merge_output()

AssertionError: 

In [ ]:
ds